In [14]:
import pandas as pd 
import numpy as np
import seaborn as sns
import plotly as plt
import data 

In [15]:
#bringing the files in to the main document to allow analysis
df_games = pd.read_csv(r'data/games.csv')
df_pff = pd.read_csv(r'data/pffScoutingData.csv')
df_players = pd.read_csv(r'data/players.csv') 
df_plays = pd.read_csv(r'data/plays.csv')
df_week1 = pd.read_csv(r'data/week1.csv')
df_week2 = pd.read_csv(r'data/week2.csv')
df_week3 = pd.read_csv(r'data/week3.csv')
df_week4 = pd.read_csv(r'data/week4.csv')
df_week5 = pd.read_csv(r'data/week5.csv')
df_week6 = pd.read_csv(r'data/week6.csv')
df_week7 = pd.read_csv(r'data/week7.csv')
df_week8 = pd.read_csv(r'data/week8.csv')


In [55]:
# offense_formation =  df_plays['offenseFormation'].value_counts()
# print(offense_formation)

SHOTGUN       5481
EMPTY         1396
SINGLEBACK    1189
I_FORM         298
PISTOL         154
JUMBO           30
WILDCAT          2
Name: offenseFormation, dtype: int64


In [66]:
#useful columns from the plays database
offensive_personnel = df_plays[df_plays.columns[25]]
defendersInTheBox = df_plays[df_plays.columns[26]]
defensive_personnel = df_plays[df_plays.columns[27]]

qb_dropback_types = df_plays[df_plays.columns[28]].value_counts()

pff_playAction = df_plays['pff_playAction']
pff_passCoverageTypes = df_plays['pff_passCoverageType']


Zone     5588
Man      2481
Other     488
Name: pff_passCoverageType, dtype: int64


In [ ]:
#Useful colums from pff_scouting 